### Packages reqd

In [ ]:
#keras
#tensorflow
#numpy
#matplotlib
#cv2

### Import Modules

In [ ]:
import tensorflow as tf
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
import os
import fnmatch
import numpy as np
#import boto3
import matplotlib.pyplot as plt
import cv2

%matplotlib inline
# import zipfile
# from subprocess import call


## Create Alexnet Model

In [ ]:
def create_model():
    #Create a sequential model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),\
     strides=(4,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(224*224*3,)))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(1,activation='linear'))
    
    
    return model

### Build the Model

In [ ]:
model = create_model()
model.summary()


### Compile the Model

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])
print("Model Compiled")

### Split Training and Validation Set

In [ ]:
def extract_x(folder):
    x = []
    folder = folder + '/'
    for root,dirs,files in os.walk(folder):
        for f in files:
            if fnmatch.fnmatch(f,"*.png"):
                img = cv2.imread(folder+f)
                img = cv2.resize(img,(224,224))
                x.append(img)
                #print(f)
    print(folder+"completed")
    lb = int(0.6*len(x))
    return np.array(x[:lb]),np.array(x[lb:])



def extract_y(folder):
    folder = folder+'/'
    y = np.load(folder+'angles1.npy')
    lb = int(0.6*len(y))
    return np.array(y[:lb]),np.array(y[lb:])

### Define Keras callbacks for accuracy improvement

In [ ]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=0,
                                  mode='min')
#ckpt = ModelCheckpoint('.model.hdf5', save_best_only=True,monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                       patience=7, verbose=1, min_delta=1e-4,
                                       mode='min')
tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=64, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None)
checkpoint = ModelCheckpoint('./saved_models/weights.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


### Train the Model

In [ ]:


    for roots,dirs,files in os.walk('.'):
         for dir in dirs:

                if fnmatch.fnmatch(dir,'batch*'):
                    train_x,valid_x = extract_x(dir)
                    train_y,valid_y = extract_y(dir)         

                    history = model.fit(train_x, train_y, batch_size=64, epochs=3,verbose=1,callbacks=[earlyStopping, reduce_lr_loss,tb,checkpoint],validation_data=(valid_x, valid_y))



### Evaluate the Model

In [ ]:
loss,mse = model.evaluate(x=valid_x, y=valid_y, batch_size=64, verbose=1, sample_weight=None, steps=None)
print("Loss:{.3f}".format(loss))